In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

print(tf.__version__)
print(keras.__version__)

1.13.1
2.2.4-tf


In [2]:
tf.enable_eager_execution()

In [3]:
learning_rate = 0.001
training_epochs = 60
batch_size = 12
img_size = 224
n_train = 2000
lr_decay_ratio = 0.05
lr_decay_epoch_num = 10
alpha = tf.constant(0.2)

tf.set_random_seed(777)

In [4]:
def _parse_function(tfrecord_serialized):
    features={'anc_image' : tf.FixedLenFeature([], tf.string),
             'pos_image' : tf.FixedLenFeature([], tf.string)}
    parsed_features = tf.parse_single_example(tfrecord_serialized, features)
    
    anc_image = tf.decode_raw(parsed_features['anc_image'], tf.uint8)
    #anc_image = parsed_features['anc_image'].numpy()
    #anc_image = tf.image.decode_jpeg(parsed_features['anc_image'])
    anc_image = tf.reshape(anc_image, [img_size, img_size, 3])
    anc_image = tf.cast(anc_image, tf.float32)/255.
    
    pos_image = tf.decode_raw(parsed_features['pos_image'], tf.uint8)
    #pos_image = tf.image.decode_jpeg(parsed_features['pos_image'])
    pos_image = tf.reshape(pos_image, [img_size, img_size, 3])
    pos_image = tf.cast(pos_image, tf.float32)/255.
    
    #neg_image_bundle = tf.decode_raw(parsed_features['neg_image_bundle'], tf.uint8)
    #neg_image_bundle = tf.image.decode_jpeg(parsed_features['neg_image_bundle'])
    #neg_image_bundle = tf.reshape(neg_image_bundle, [15, img_size, img_size, 3])
    #neg_image_bundle = tf.cast(neg_image_bundle, tf.float32)/255.
    
    return anc_image, pos_image
    
    

In [5]:
from tensorflow.keras.layers import Input, Conv2D, Dense, ReLU, BatchNormalization, MaxPool2D, GlobalAveragePooling2D, Concatenate, Lambda

In [6]:
def create_model():
    inputs = Input(shape=(img_size, img_size, 3))
    net = Conv2D(64, 7, 2, 'SAME')(inputs)
    net = BatchNormalization()(net)
    net = ReLU()(net)
    net = MaxPool2D(pool_size=3, strides=2, padding='SAME')(net)
    net = Conv2D(64, 1, 1, 'SAME')(net)
    net = BatchNormalization()(net)
    net = ReLU()(net)
    net = Conv2D(192, 3, 1, 'SAME')(net)
    net = BatchNormalization()(net)
    net = ReLU()(net)
    net = MaxPool2D(3, 2, 'SAME')(net)
    #####3a
    a_a = Conv2D(64, 1, 1, 'SAME')(net)
    a_b = Conv2D(96, 1, 1, 'SAME')(net)
    a_b = Conv2D(128, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(16, 1, 1, 'SAME')(net)
    a_c = Conv2D(32, 5, 1, 'SAME')(a_c)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(32, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    net = BatchNormalization()(net)
    net = ReLU()(net)

    ######3b
    a_a = Conv2D(64, 1, 1, 'SAME')(net)
    a_b = Conv2D(96, 1, 1, 'SAME')(net)
    a_b = Conv2D(128, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(64, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 3c
    a_b = Conv2D(128, 1, 1, 'SAME')(net)
    a_b = Conv2D(256, 3, 2, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 2, 'SAME')(a_c)
    a_d = MaxPool2D(3, 2, 'SAME')(net)
    a_d = Conv2D(320, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_b, a_c, a_d])
    
    ###### 4a
    a_a = Conv2D(256, 1, 1, 'SAME')(net)
    a_b = Conv2D(96, 1, 1, 'SAME')(net)
    a_b = Conv2D(192, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 4b
    a_a = Conv2D(224, 1, 1, 'SAME')(net)
    a_b = Conv2D(112, 1, 1, 'SAME')(net)
    a_b = Conv2D(224, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 4c
    a_a = Conv2D(192, 1, 1, 'SAME')(net)
    a_b = Conv2D(128, 1, 1, 'SAME')(net)
    a_b = Conv2D(256, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 4d
    a_a = Conv2D(160, 1, 1, 'SAME')(net)
    a_b = Conv2D(144, 1, 1, 'SAME')(net)
    a_b = Conv2D(288, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(32, 1, 1, 'SAME')(net)
    a_c = Conv2D(64, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 4e    
    a_b = Conv2D(160, 1, 1, 'SAME')(net)
    a_b = Conv2D(256, 3, 2, 'SAME')(a_b)
    a_c = Conv2D(64, 1, 1, 'SAME')(net)
    a_c = Conv2D(128, 5, 2, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 2, 'SAME')(net)
    a_d = Conv2D(640, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_b, a_c, a_d])
    
    ###### 5a
    a_a = Conv2D(384, 1, 1, 'SAME')(net)
    a_b = Conv2D(192, 1, 1, 'SAME')(net)
    a_b = Conv2D(384, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(48, 1, 1, 'SAME')(net)
    a_c = Conv2D(128, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### 5b
    a_a = Conv2D(384, 1, 1, 'SAME')(net)
    a_b = Conv2D(192, 1, 1, 'SAME')(net)
    a_b = Conv2D(384, 3, 1, 'SAME')(a_b)
    a_c = Conv2D(48, 1, 1, 'SAME')(net)
    a_c = Conv2D(128, 5, 1, 'SAME')(a_c)
    #a_d = Lambda(lambda x: l2_norm_pooling(x))(net)
    a_d = MaxPool2D(3, 1, 'SAME')(net)
    a_d = Conv2D(128, 1, 1, 'SAME')(a_d)
    net = Concatenate()([a_a, a_b, a_c, a_d])
    
    ###### avg pool and fully conn
    net = GlobalAveragePooling2D()(net)
    net = Dense(units=128)(net)
    

    return keras.Model(inputs=inputs, outputs=net)

In [7]:
model = create_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 112, 112, 64) 0           batch_normalization_v1[0][0]     
_____________________________________

In [8]:
def loss_fn(model, anc_image, pos_image):
    logit_anc = model(anc_image, training=True)
    logit_pos = model(pos_image, training=True)
    
    print('logit_anc1 = ' + str(logit_anc))
    #[12, 128]
    logit_anc = tf.div(logit_anc, tf.math.sqrt(tf.reduce_sum(tf.square(logit_anc), 1, keepdims=True)))    
    print('logit_anc2 = ' + str(logit_anc))
    #[12, 128]
    logit_pos = tf.div(logit_pos, tf.math.sqrt(tf.reduce_sum(tf.square(logit_pos), 1, keepdims=True)))
    print('logit_pos = ' + str(logit_pos))
    
    neg_list=[]
    for idx, anc_tensor in enumerate(logit_anc):
        minValue = tf.constant(100, tf.float32)
        index = 0
        minTensor = []
        for i in range(len(logit_anc)):
            if i!=idx:
                #anc_tensor=[128,] logit_anc[i]=[128,]
                if minValue>tf.math.minimum(get_distance(anc_tensor, logit_anc[i]), get_distance(anc_tensor, logit_pos[i])):                    
                    if tf.math.greater(get_distance(anc_tensor, logit_anc[i]), get_distance(anc_tensor, logit_pos[i])):
                        minValue = get_distance(anc_tensor, logit_anc[i])
                        minTensor = logit_anc[i]
                    else:
                        minValue = get_distance(anc_tensor, logit_pos[i])
                        minTensor = logit_pos[i]
                #print("idx = " + str(idx) + ", i = " + str(i) + "value = " + str(tf.math.minimum(get_distance(anc_tensor, logit_anc[i]), get_distance(anc_tensor, logit_pos[i]))))
            
        #print(minValue)
        neg_list.append(minTensor)
    #[12, 128]
    neg_list = tf.convert_to_tensor(neg_list)
    #print("neg_list = " + str(neg_list))
        
        
    #neg = tf.reshape(neg, (15, img_size, img_size, 3))
    
    #logit_neg = model(neg, training=True)
    
    #logit_neg = tf.stack(neg_list)
    
    pos_dist = tf.math.sqrt(tf.reduce_sum(tf.square(tf.subtract(logit_anc, logit_pos)), 1))
    neg_dist = tf.math.sqrt(tf.reduce_sum(tf.square(tf.subtract(logit_anc, neg_list)), 1))
    print('post_dist = ' + str(pos_dist))
    print('neg_dist = ' + str(neg_dist))
    
    #이 부분 다시 봐야 함 논문 참고해야 함
    temp_sub = tf.subtract(pos_dist, neg_dist)
    print('temp_min1 = ' + str(tf.math.abs(temp_sub)))
    #temp_min = -tf.reduce_min(tf.math.abs(temp_sub))
    #print('temp_min2 = ' + str(temp_min))
    
    #print(tf.add(temp_min, alpha))
    
    
    return tf.reduce_mean(tf.add(temp_sub, alpha))
    

In [9]:
def grad(model, anc_image, pos_image):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, anc_image, pos_image)
    return tape.gradient(loss, model.variables), loss

In [10]:
def get_distance(tensor1, tensor2):
    return tf.reduce_sum(tf.square(tf.subtract(tensor1, tensor2)), 0)

In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [12]:
data_list = os.listdir('D:\\facenet\\data\\tfrecord')
print(data_list)

train_tfr_path = 'D:\\facenet\\data\\tfrecord\\facenet_data_0000'
train_dataset = tf.data.TFRecordDataset(train_tfr_path)
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=8)
train_dataset = train_dataset.shuffle(buffer_size=n_train*2).prefetch(buffer_size=batch_size).batch(batch_size)

['facenet_data_0000', 'facenet_data_0001', 'facenet_data_0002', 'facenet_data_0003', 'facenet_data_0004', 'facenet_data_0005', 'facenet_data_0006', 'facenet_data_0007', 'facenet_data_0008', 'facenet_data_0009', 'facenet_data_0010', 'facenet_data_0011', 'facenet_data_0012', 'facenet_data_0013', 'facenet_data_0014', 'facenet_data_0015', 'facenet_data_0016', 'facenet_data_0017', 'facenet_data_0018', 'facenet_data_0019', 'facenet_data_0020', 'facenet_data_0021', 'facenet_data_0022', 'facenet_data_0023', 'facenet_data_0024', 'facenet_data_0025', 'facenet_data_0026', 'facenet_data_0027', 'facenet_data_0028', 'facenet_data_0029', 'facenet_data_0030', 'facenet_data_0031', 'facenet_data_0032', 'facenet_data_0033', 'facenet_data_0034', 'facenet_data_0035', 'facenet_data_0036', 'facenet_data_0037', 'facenet_data_0038', 'facenet_data_0039', 'facenet_data_0040', 'facenet_data_0041', 'facenet_data_0042', 'facenet_data_0043', 'facenet_data_0044', 'facenet_data_0045', 'facenet_data_0046', 'facenet_dat

In [13]:
for epoch in range(training_epochs):
    avg_loss = 0
    train_step = 0
    
    for record in data_list:
        train_tfr_path = os.path.join('D:\\facenet\\data\\tfrecord', record)
        train_dataset = tf.data.TFRecordDataset(train_tfr_path)
        train_dataset = train_dataset.map(_parse_function, num_parallel_calls=8)
        train_dataset = train_dataset.shuffle(buffer_size=n_train*2).prefetch(buffer_size=batch_size).batch(batch_size)    
    
        for anc_image, pos_image in train_dataset:        
            grads, loss = grad(model, anc_image, pos_image)
            optimizer.apply_gradients(zip(grads, model.variables))
            #loss = loss_fn(model, anc_image, pos_image, neg_image_bundle)
            avg_loss = avg_loss + loss
            train_step += 1
            if train_step%20000==0:
                print(train_step)
    avg_loss = avg_loss / train_step
    
    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss))
    model.save_weights('D:\\facenet\\weights\\my_model_{}.h5'.format(epoch), save_format='h5')

logit_anc1 = tf.Tensor(
[[ 0.7738468  -2.418384    6.799861   ... -4.0212374  -0.4459497
   3.7735775 ]
 [ 1.4125979  -2.8265777   9.218908   ... -2.3362315   2.0223987
   4.916623  ]
 [ 1.8678046  -0.21955967  6.8159122  ... -4.0034437   1.3920453
   4.8235483 ]
 ...
 [ 1.8517503  -2.1988049   9.8580475  ... -4.566474    1.6829486
   5.4590507 ]
 [-0.40790188 -3.407509    4.3796706  ... -3.3384974  -0.06262726
   2.9986508 ]
 [ 0.98731554 -1.7045496   6.5643244  ... -3.9814937   0.9916187
   4.095968  ]], shape=(12, 128), dtype=float32)
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
logit_anc2 = tf.Tensor(
[[ 0.02533866 -0.07918702  0.2226531  ... -0.13167049 -0.01460208
   0.12356117]
 [ 0.03563284 -0.07130053  0.23254731 ... -0.05893153  0.05101509
   0.12402201]
 [ 0.05840284 -0.00686523  0.21312113 ... -0.12518038  0.04352671
   0.15082355]
 ...
 [ 0.04607723 -0.05471302  0.24529849 ... -0.1136279   0.04187693
   0.13583794]
 [-0.01682252 -0.14053106

logit_anc1 = tf.Tensor(
[[-18818.514  -23666.8    -18385.244  ...  16759.084  -21444.842
    1938.7008]
 [-20612.398  -25958.729  -19914.32   ...  18584.205  -23527.127
    1960.9572]
 [-30449.729  -38585.76   -29243.877  ...  27458.97   -34691.6
    2735.7117]
 ...
 [-16870.105  -21198.479  -16321.421  ...  15221.845  -19279.951
    1695.2599]
 [-13540.048  -16911.559  -13150.19   ...  12236.93   -15494.124
    1416.0631]
 [-29606.87   -37354.9    -28501.     ...  26685.195  -33751.184
    2811.1843]], shape=(12, 128), dtype=float32)
logit_anc2 = tf.Tensor(
[[-0.10814046 -0.13600111 -0.10565068 ...  0.09630597 -0.12323264
   0.01114073]
 [-0.10793266 -0.13592763 -0.10427731 ...  0.09731243 -0.12319504
   0.01026816]
 [-0.10803422 -0.13690048 -0.10375591 ...  0.09742315 -0.12308419
   0.00970618]
 ...
 [-0.10790033 -0.13558438 -0.10439097 ...  0.09735814 -0.12331358
   0.0108428 ]
 [-0.10781722 -0.13466401 -0.10471285 ...  0.0974407  -0.12337721
   0.01127588]
 [-0.10802741 -0.13629785

logit_anc1 = tf.Tensor(
[[-275250.78  -298642.75  -248319.05  ...  245493.    -300544.9
    52066.598]
 [-224204.88  -242978.    -202488.7   ...  200009.6   -245552.81
    43355.99 ]
 [-364858.    -395084.88  -328380.62  ...  325817.5   -398571.75
    69648.64 ]
 ...
 [-332042.25  -358472.38  -299126.7   ...  296527.78  -363340.78
    64942.76 ]
 [-336832.78  -364110.94  -303994.25  ...  299780.    -367534.5
    64786.91 ]
 [-311292.56  -337029.9   -280957.44  ...  277061.7   -339791.72
    59874.48 ]], shape=(12, 128), dtype=float32)
logit_anc2 = tf.Tensor(
[[-0.11095612 -0.12038563 -0.10009969 ...  0.09896048 -0.12115241
   0.02098852]
 [-0.11087633 -0.12016023 -0.100137   ...  0.09891101 -0.12143356
   0.02144089]
 [-0.11094792 -0.12013946 -0.09985569 ...  0.09907628 -0.12119977
   0.02117912]
 ...
 [-0.11094438 -0.1197754  -0.09994639 ...  0.09907802 -0.12140206
   0.02169915]
 [-0.11099791 -0.119987   -0.10017649 ...  0.09878775 -0.12111517
   0.0213495 ]
 [-0.11102372 -0.12020304

logit_anc1 = tf.Tensor(
[[-1268224.2  -1289059.   -1103926.2  ...  1120317.2  -1352056.
    261575.73]
 [-1029770.3  -1039339.9   -889343.75 ...   914612.   -1100559.2
    221106.44]
 [-1501402.2  -1519750.5  -1299448.5  ...  1329617.   -1601594.8
    316795.88]
 ...
 [-1126430.4  -1138393.    -973748.8  ...   999623.5  -1203888.5
    241451.69]
 [ -812268.1   -822465.9   -702154.44 ...   719950.06  -867373.06
    172288.75]
 [ -842328.4   -851667.4   -727832.4  ...   748060.4   -900784.1
    180825.89]], shape=(12, 128), dtype=float32)
logit_anc2 = tf.Tensor(
[[-0.11265624 -0.11450699 -0.09806167 ...  0.09951767 -0.12010301
   0.02323575]
 [-0.11269912 -0.11374643 -0.0973307  ...  0.10009608 -0.12044634
   0.02419812]
 [-0.11268845 -0.11406559 -0.09753072 ...  0.09979503 -0.12020845
   0.02377726]
 ...
 [-0.11271331 -0.11391032 -0.09743563 ...  0.10002472 -0.12046396
   0.02416023]
 [-0.11268725 -0.11410199 -0.097411   ...  0.09987981 -0.12033204
   0.02390189]
 [-0.1126532  -0.113902

logit_anc1 = tf.Tensor(
[[-2609781.5  -2553419.   -2208724.2  ...  2304867.5  -2752679.8
    569919.9 ]
 [-2378119.5  -2325339.5  -2011818.1  ...  2104850.8  -2514530.
    525903.25]
 [-1942445.6  -1901689.4  -1645540.   ...  1717779.2  -2055316.9
    430250.5 ]
 ...
 [-1775745.5  -1737098.4  -1503501.8  ...  1568871.8  -1872014.9
    382347.53]
 [-2829460.   -2773563.8  -2406443.   ...  2492862.   -2978116.5
    595130.06]
 [-1991602.2  -1958154.   -1698437.2  ...  1751168.2  -2101178.8
    430924.3 ]], shape=(12, 128), dtype=float32)
logit_anc2 = tf.Tensor(
[[-0.11366041 -0.11120573 -0.09619369 ...  0.10038089 -0.11988387
   0.02482098]
 [-0.1136352  -0.11111318 -0.09613199 ...  0.10057742 -0.12015339
   0.02512957]
 [-0.11358465 -0.11120143 -0.09622307 ...  0.10044727 -0.1201848
   0.02515893]
 ...
 [-0.11367325 -0.11119927 -0.09624574 ...  0.10043035 -0.11983587
   0.02447574]
 [-0.11363043 -0.11138565 -0.09664217 ...  0.10011274 -0.11960044
   0.02390028]
 [-0.11359129 -0.11168357

...

 0.00075761 0.00041808 0.00065578 0.00129628 0.00161227 0.00122074], shape=(12,), dtype=float32)
neg_dist = tf.Tensor(
[0.00102199 0.00307048 0.00063004 0.00195938 0.00153277 0.00088036
 0.00147381 0.00167462 0.00158375 0.0022631  0.00192817 0.00332485], shape=(12,), dtype=float32)
temp_min1 = tf.Tensor(
[0.00010427 0.00262091 0.00161181 0.00141666 0.00060189 0.00016118
 0.0007162  0.00125655 0.00092797 0.00096681 0.0003159  0.00210411], shape=(12,), dtype=float32)
logit_anc1 = tf.Tensor(
[[-9.7532587e+15 -1.0004148e+16 -1.1299169e+16 ...  1.9049821e+15
  -6.6401789e+15 -1.3769787e+15]
 [-3.0892261e+15 -3.1798349e+15 -3.5732013e+15 ...  6.1971264e+14
  -2.1340230e+15 -4.3249160e+14]
 [-4.1958974e+15 -4.3115547e+15 -4.8572590e+15 ...  8.2305331e+14
  -2.8761646e+15 -5.8347949e+14]
 ...
 [-4.1558869e+15 -4.2723299e+15 -4.8099607e+15 ...  8.1746354e+14
  -2.8530056e+15 -5.7670029e+14]
 [-4.0820848e+15 -4.1962856e+15 -4.7265857e+15 ...  8.0275502e+14
  -2.7988557e+15 -5.6776810e+14]
 [-4.4

logit_anc1 = tf.Tensor(
[[-5.54024475e+15 -5.70736622e+15 -6.41826330e+15 ...  1.10356024e+15
  -3.81181945e+15 -7.65716366e+14]
 [-4.38360468e+15 -4.50065543e+15 -5.08170276e+15 ...  8.52008232e+14
  -2.98360157e+15 -6.09505788e+14]
 [-5.66345662e+15 -5.83186121e+15 -6.55917742e+15 ...  1.12855379e+15
  -3.89557238e+15 -7.84584896e+14]
 ...
 [-4.84429952e+15 -4.98279879e+15 -5.60989945e+15 ...  9.54265229e+14
  -3.32267528e+15 -6.68625141e+14]
 [-6.90714225e+15 -7.10571684e+15 -8.00535304e+15 ...  1.36163321e+15
  -4.72853136e+15 -9.57682278e+14]
 [-4.37841583e+15 -4.50894096e+15 -5.06985885e+15 ...  8.73094374e+14
  -3.01414953e+15 -6.06104443e+14]], shape=(12, 128), dtype=float32)
logit_anc2 = tf.Tensor(
[[-0.1188171  -0.12240122 -0.13764724 ...  0.02366715 -0.08174897
  -0.01642169]
 [-0.118898   -0.1220728  -0.13783275 ...  0.02310931 -0.08092523
  -0.01653183]
 [-0.11887357 -0.12240832 -0.13767438 ...  0.02368787 -0.08176643
  -0.01646811]
 ...
 [-0.11888853 -0.12228758 -0.137677

logit_anc1 = tf.Tensor(
[[-3.7055437e+15 -3.8170190e+15 -4.2923597e+15 ...  7.3927903e+14
  -2.5485965e+15 -5.1033050e+14]
 [-7.0622324e+15 -7.2768171e+15 -8.1907807e+15 ...  1.3922496e+15
  -4.8404604e+15 -9.6185598e+14]
 [-5.3640437e+15 -5.5152813e+15 -6.2095563e+15 ...  1.0656840e+15
  -3.6798319e+15 -7.4545976e+14]
 ...
 [-2.6492207e+15 -2.7310586e+15 -3.0711856e+15 ...  5.2646135e+14
  -1.8210396e+15 -3.6167369e+14]
 [-3.2114166e+15 -3.3065614e+15 -3.7262304e+15 ...  6.3018525e+14
  -2.1948323e+15 -4.3940475e+14]
 [-7.0462525e+15 -7.2447175e+15 -8.1600223e+15 ...  1.3929490e+15
  -4.8278525e+15 -9.7502160e+14]], shape=(12, 128), dtype=float32)
logit_anc2 = tf.Tensor(
[[-0.11883456 -0.1224095  -0.1376534  ...  0.02370823 -0.08173196
  -0.01636599]
 [-0.11880909 -0.12241908 -0.13779484 ...  0.02342204 -0.08143185
  -0.01618146]
 [-0.1189083  -0.12226088 -0.13765134 ...  0.02362372 -0.08157326
  -0.0165251 ]
 ...
 [-0.11882704 -0.12249776 -0.13775368 ...  0.02361368 -0.08168015
  -0.

logit_anc1 = tf.Tensor(
[[-4.2307248e+15 -4.3634758e+15 -4.9103856e+15 ...  8.3877745e+14
  -2.9000457e+15 -5.7682337e+14]
 [-3.9787788e+15 -4.1029130e+15 -4.6156714e+15 ...  7.9042350e+14
  -2.7306065e+15 -5.4334964e+14]
 [-4.5974361e+15 -4.7354355e+15 -5.3355117e+15 ...  9.0399492e+14
  -3.1423165e+15 -6.2867315e+14]
 ...
 [-8.7886756e+15 -9.0540348e+15 -1.0205976e+16 ...  1.7384032e+15
  -6.0006352e+15 -1.2149763e+15]
 [-4.7654203e+15 -4.9121917e+15 -5.5267580e+15 ...  9.4792170e+14
  -3.2712753e+15 -6.5335680e+14]
 [-3.9148992e+15 -4.0401555e+15 -4.5415333e+15 ...  7.8340311e+14
  -2.6929826e+15 -5.3568990e+14]], shape=(12, 128), dtype=float32)
logit_anc2 = tf.Tensor(
[[-0.11878221 -0.12250935 -0.13786444 ...  0.02354959 -0.08142195
  -0.01619495]
 [-0.11882517 -0.12253241 -0.1378458  ...  0.02360579 -0.08154884
  -0.01622699]
 [-0.11882139 -0.12238801 -0.13789706 ...  0.02336388 -0.08121362
  -0.01624815]
 ...
 [-0.11883679 -0.12242486 -0.13800094 ...  0.02350596 -0.08113807
  -0.

KeyboardInterrupt: 